## Exercise 4: Introduction to ML - White-box ML with Symbolic Regression
We are going to play with a trivial test case of regression, using a Symbolic Regression library called PySR. As the authors of PySR apparently love pain, they decided to write their library in Julia and then create Python bindings. Expect to see some really weird outputs while executing the next two cells, but there is a high probability everything will be fine in the end.

In [ ]:
!pip install pysr

In [ ]:
import pysr

import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split

Now, let's create a simple dataset with five features $X_0, ... X_5$, and one output variable, y, where:

$y = 2.5382 \cdot \cos(X_3) + (X_0)^2 - 2$

This run will last a while, Symbolic Regression can be quite slow when compared to other algorithms like Random Forest. The upside is that the best equation obtained will be perfectly human readable.

In [ ]:
# generate dataset
np.random.seed(0)
X = 2 * np.random.randn(100, 5)
y = 2.5382 * np.cos(X[:, 3]) + X[:, 0] ** 2 - 2

fig, ax = plt.subplots()
ax.scatter([i for i in range(0, 100)], y)

In [ ]:
default_pysr_params = dict(
    populations=30,
    model_selection="best",
    verbosity=0,
    progress=False,
)

model = PySRRegressor(
    niterations=30,
    binary_operators=["+", "*"],
    unary_operators=["cos", "exp", "sin"],
    **default_pysr_params,
)

model.fit(X, y)

print("Best compromise obtained:", model.sympy())

It worked really well! What happens if we now tackle an example with some noise?

In [ ]:
noise = np.random.randn(100) * 0.1
y = y + noise

model.fit(X, y)

Interestingly, PySR does not return just a single equation, but a series of compromises between complexity and fitting. The added noise might have led the algorithm to develop extremely complex equations that also fit part of the noise. Let's check all the equations found in the end.

In [ ]:
for equation in model.equations_["sympy_format"] :
  print(equation)

We can see equations that remind us of the correct one, but their terms are slightly different, and other (irrelevant) variables have been added to the equations, just because they randomly fit the noise.

Let's take a look at the Pareto front of complexity vs fitting. Each point will represent a candidate equation.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(model.equations_["complexity"], model.equations_["loss"])
ax.set_xlabel("complexity")
ax.set_ylabel("error")
ax.set_title("Pareto front of equations, complexity vs fitting")

It's easy to notice that, from left to right, the equations keep increasing in complexity, while only very slightly reducing error. The equation considered the 'best' is picked by a heuristic on the Pareto front, but it seems to work quite well even with some noise.



In [ ]:
print(model.sympy())

# find the index of the equation in the Pareto front; it's a bit of convoluted code, but trust me on this
index = model.equations_[model.equations_["sympy_format"] == model.sympy()].index.tolist()[0]

fig, ax = plt.subplots()
ax.scatter(model.equations_["complexity"], model.equations_["loss"])
ax.scatter(model.equations_["complexity"].iloc[index], model.equations_["loss"].iloc[index], color='red', label="Model considered the best")
ax.set_xlabel("complexity")
ax.set_ylabel("error")
ax.set_title("Pareto front of equations, complexity vs fitting")
ax.legend(loc="best")
